In [1]:
from __future__ import print_function
import h5py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input, Concatenate, Reshape
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.utils import plot_model
from keras.models import Model
from keras.optimizers import adadelta as ada
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import copy
import pydot
from keras.utils import multi_gpu_model
from keras import backend as K



import gc

import time



%matplotlib inline

Using TensorFlow backend.
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\mrathbun2018\.conda\envs\mattwork\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\mrathbun2018\.con

In [2]:
hdf5_path = 'Data/dataset.hdf5'
subtract_mean = True
hdf5_file = h5py.File(hdf5_path, "r")
if subtract_mean:
    mm = hdf5_file["train_mean"][0, ...]
    mm = mm[np.newaxis, ...]
data_num = hdf5_file["train_flow"].shape[0]
    

    
num_classes = 2
epochs = 30

flow_rows, flow_cols = 298, 17

    

x_train = hdf5_file["train_flow"][:,...]
if subtract_mean:
    x_train -= mm

y_train = hdf5_file["train_labels"][:, ...]

hdf5_file.close()
    
hdf5_path = 'Data/dataset-IoT.hdf5'
hdf5_file = h5py.File(hdf5_path, "r")

x_test = hdf5_file["IoT_flow"][:,...]
if subtract_mean:
    x_test -= mm

y_test = hdf5_file["labels"][:, ...]

hdf5_file.close()





class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
d_class_weights = dict(enumerate(class_weights))

packets = 298
x_train = x_train[:,:packets,:,:]
x_test = x_test[:,:packets,:,:]

input_shape = (x_train.shape[1], x_train.shape[2], x_train.shape[3])


y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [3]:
AccScores = []
Times = []
LossScores = []
#AucScores = []
#F1Scores = []
#PrecisionScores = []
t2 = time.time()
for i in range (1,50):
    batch_size = 1024
    epochs = 30
    filters= 32
    kernel_size= (3,3)
    activations= "sigmoid"
    pool_size= (2,2)
    dropout = 0.29104403739531765
    lr = 0.001433773291970261
    rmsprop = keras.optimizers.RMSprop(lr=lr)
    optim = rmsprop
    layers = 2
    model = Sequential()

    model.add(Conv2D(filters, kernel_size=kernel_size,activation=activations, input_shape=input_shape,padding = "same"))
    for i in range(layers-1):
        model.add(Conv2D(filters,kernel_size=kernel_size, activation=activations, padding = "valid"))
        
    model.add(MaxPooling2D(pool_size=(pool_size)))
    model.add(Flatten())
    model.add(Dropout(dropout))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(num_classes, activation='softmax'))
    #model.summary()
    try:
        model = multi_gpu_model(model, gpus = 4)
    except:
        pass
    model.compile(loss='binary_crossentropy', optimizer=optim, metrics=['accuracy'])
    t0 = time.time()
    model.fit(x_train,y_train, batch_size=batch_size, epochs=epochs, verbose=0, class_weight=class_weights, shuffle=True)
    scores = model.evaluate(x_test, y_test, verbose=0)
    t1 = time.time()
    total_run = t1-t0
    Times.append(total_run)
    print("%s: %.5f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s: %.5f%%" % (model.metrics_names[0], scores[0]))
    print('Fit and Evaluate Time:',total_run)
    AccScores.append(scores[1] * 100)
    LossScores.append(scores[0])
    del model
    gc.collect()
    K.clear_session()
    gc.collect()
    
print("%.5f%% (+/- %.5f%%)" % (np.mean(AccScores), np.std(AccScores)))
print("%.5f%% (+/- %.5f%%)" % (np.mean(LossScores), np.std(LossScores)))
print("%.5f%% (+/- %.5f%%)" % (np.mean(Times), np.std(Times)))
t3 = time.time()
total_time = t3-t2
print('Time to do everything:',total_time)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

accuracy: 87.46887%
loss: 0.27770%
Fit and Evaluate Time: 89.2846999168396
accuracy: 87.56374%
loss: 0.29504%
Fit and Evaluate Time: 85.84562397003174
accuracy: 88.00009%
loss: 0.26786%
Fit and Evaluate Time: 85.7653968334198
accuracy: 87.76768%
loss: 0.26710%
Fit and Evaluate Time: 85.18278884887695
accuracy: 83.30685%
loss: 0.28681%
Fit and Evaluate Time: 82.3373634815216
accuracy: 87.16295%
loss: 0.28182%
Fit and Evaluate Time: 81.96373224258423
accuracy: 87.69891%
loss: 0.28215%
Fit and Evaluate Time: 82.48232412338257
accuracy: 83.33057%
loss: 0.29151%
Fit and Evaluate Time: 86.96925973892212
accuracy: 87.45939%
loss: 0.27482%
Fit and Evaluate Time: 88.251056432724
accuracy: 87.82697%
loss: 0.26559%
Fit and Evaluate Time: 86.57623839378357
accuracy: 87.97638%
loss: 0.26512%
Fit and Evaluate Time: 87.72254848480225
accuracy: 87.53765%
loss: 0.28136%
Fit and Evaluate Time: 81.793133020401

In [4]:
AccScores

[87.46887445449829,
 87.56373524665833,
 88.00009489059448,
 87.76768445968628,
 83.30684900283813,
 87.16294765472412,
 87.69891262054443,
 83.33056569099426,
 87.45938539505005,
 87.82697319984436,
 87.97637820243835,
 87.53764629364014,
 87.12263107299805,
 88.5550320148468,
 83.70052576065063,
 87.96452283859253,
 87.77480125427246,
 87.73211240768433,
 83.2119882106781,
 88.18507194519043,
 82.94163942337036,
 87.74634003639221,
 82.94163942337036,
 82.94163942337036,
 82.94163942337036,
 87.77005672454834,
 86.96373701095581,
 87.56610751152039,
 86.91156506538391,
 87.35504150390625,
 88.3676826953888,
 88.71392607688904,
 87.60642409324646,
 88.61194849014282,
 87.50919103622437,
 82.94163942337036,
 87.9929780960083,
 87.40246891975403,
 88.01432251930237,
 84.35506224632263,
 86.76690459251404,
 82.94163942337036,
 82.94163942337036,
 87.68231272697449,
 86.97322607040405,
 87.28152513504028,
 87.26018071174622,
 82.94163942337036,
 87.73685693740845]

In [5]:
LossScores

[0.27769745134200086,
 0.29503988322422464,
 0.2678590508051409,
 0.26710454454858384,
 0.2868086357948651,
 0.2818233267852429,
 0.28215008380884216,
 0.29150747080409795,
 0.274819421176835,
 0.2655889592109057,
 0.26511989357827426,
 0.28136088573315216,
 0.3034818392171602,
 0.2619396248918438,
 0.29319895227982434,
 0.2808526711024304,
 0.28680695181275506,
 0.27588431302221367,
 0.2996846738774524,
 0.2905991410383671,
 2.6234215325208003,
 0.26187385342304714,
 2.6234215325208003,
 2.6234215325208003,
 0.32914232458174264,
 0.2623940799537706,
 0.2828748356702446,
 0.2823331882181456,
 0.29228865190134345,
 0.2978966362972444,
 0.26378162383908105,
 0.2675153765194241,
 0.2729670757454183,
 0.2758194258364603,
 0.2883056764407791,
 2.6234215325208003,
 0.26178466158801955,
 0.27729379129241016,
 0.2646817768598537,
 0.3140340435181072,
 0.2765410473447382,
 2.6234215325208003,
 0.3397910925234475,
 0.26165526527971944,
 0.30061166674435136,
 0.2928989733259825,
 0.29638089413215

In [6]:
Times

[89.2846999168396,
 85.84562397003174,
 85.7653968334198,
 85.18278884887695,
 82.3373634815216,
 81.96373224258423,
 82.48232412338257,
 86.96925973892212,
 88.251056432724,
 86.57623839378357,
 87.72254848480225,
 81.793133020401,
 86.80074119567871,
 84.1903932094574,
 85.05938649177551,
 88.1765387058258,
 82.43021154403687,
 85.98049712181091,
 87.22051954269409,
 86.06084418296814,
 82.94434452056885,
 83.58836555480957,
 85.22335481643677,
 81.92413783073425,
 88.03998899459839,
 86.66629576683044,
 86.14387488365173,
 82.16863965988159,
 82.56818985939026,
 88.14147138595581,
 86.53589868545532,
 88.09033203125,
 87.84896659851074,
 86.21457362174988,
 82.36671257019043,
 87.39039421081543,
 82.50909304618835,
 86.76426243782043,
 85.48515033721924,
 88.8928632736206,
 83.06658959388733,
 81.9048068523407,
 82.28714609146118,
 82.84924268722534,
 84.15320229530334,
 88.1617500782013,
 86.88023114204407,
 87.0988073348999,
 82.7114028930664]